### (1) 라이브러리 설치

In [ ]:
#region install library
!pip install ipywidgets
!pip install pymysql
!pip install cryptography
#endregion

### (2) DB 목록 초기화, 변경할 DB 선택

(😀)여기 db list 붙여넣기 해주세요 

code cell 에서 ctal+a -> ctal+v

In [ ]:
#region 
db_list = {
    # example
    # "db_name_1" : {
    #       "host": "HOST_URL",
    #       "port": 3306
    # },
    # "db_name_2" : {
    #       "host": "HOST_URL",
    #       "port": 3306
    # },
}
#endregion

#### ------------------------------------------------------------------------------------------

In [ ]:
# @hide_code
#region init
import ipywidgets as widgets

md_file = "secret.md"
md_db_list = {}
try:
    # 파일 읽기
    with open(md_file, "r") as f:
        markdown_content = f.read()
        lines = markdown_content.strip().split("\n")
        if len(lines) > 2:
            header = lines[0]
            for row in lines[2:]:
                values = row.split("|")
                if len(values) == 8:  # 예상되는 테이블 형식에 맞는지 확인
                    name = values[1].strip()
                    host = values[2].strip()
                    port = values[3].strip()
                    username = values[4].strip()
                    password = values[5].strip()
                    update_date = values[6].strip()
                    md_db_list[name] = {"host": host, "port": port, "username": username, "password": password,
                                        "update_date": update_date}
except (Exception,):
    pass

# 체크박스들을 담을 리스트 생성
checkboxes = []
sorted_db_keys = sorted(db_list.keys())
# 체크박스 생성 및 리스트에 추가
for k in sorted_db_keys:
    checkbox = widgets.Checkbox(value=True, description=k)
    checkboxes.append(checkbox)

# "Select All" 체크박스 생성
select_all_checkbox = widgets.Checkbox(value=True, description='Select All')


# "Select All" 체크박스의 값이 변경될 때 호출될 함수 정의
def select_all_change(change):
    for checkbox in checkboxes:
        checkbox.value = change.new


# "Select All" 체크박스의 값이 변경되면 select_all_change 함수 호출
select_all_checkbox.observe(select_all_change, 'value')

# 체크박스들을 그리드로 배치
checkbox_grid = widgets.GridBox(checkboxes, layout=widgets.Layout(grid_template_columns="repeat(3, auto)"))

# "Select All" 체크박스와 체크박스 그리드를 세로로 배치
widgets.VBox([select_all_checkbox, checkbox_grid])

#endregion

### (3) 계정 정보

In [ ]:
# @hide_code
#region input account
target_db_list = {}
for checkbox in checkboxes:
    if not checkbox.value:
        continue
    if checkbox.description in db_list:
        target_db_list[checkbox.description] = db_list[checkbox.description]

if md_db_list:
    exists_md = True
    new_password = input(f"new password: ")
    print("new password: ", new_password)
else:
    exists_md = False
    username = input(f"current username: ")
    password = input(f"current password: ")
    new_password = input(f"new password: ")

    print("current username: ", username)
    print("current password: ", password)
    print("new password: ", new_password)

#endregion

### (4) 비밀번호 변경 프로세스

In [ ]:
# @hide_code
#region change passwords
from IPython.display import display, Markdown
import pymysql
import datetime


def change_password(db_info: dict, username: str, password: str, new_password: str):
    try:
        with pymysql.connect(host=db_info["host"], port=db_info["port"], user=username, password=password,
                             connect_timeout=3) as connection:
            with connection.cursor() as cursor:
                query = f"ALTER USER '{username}'@'%' IDENTIFIED BY '{new_password}'"
                cursor.execute(query)
            connection.commit()
            return True, ""
    except (Exception,) as e:
        return False, repr(e)


result_db_list = {}
for k, v in target_db_list.items():
    if exists_md:
        username = md_db_list[k]["username"]
        password = md_db_list[k]["password"]

    success, err_msg = change_password(db_info=v, username=username, password=password, new_password=new_password)
    if success:
        print(f"{k} 비밀번호 변경 성공 !!")
    else:
        print(f"{k} 비밀번호 변경 실패 .. {err_msg}")

    if k in md_db_list and not success:
        update_date = md_db_list[k]["update_date"]
    elif not success:
        update_date = ""
    else:
        update_date = datetime.date.today().strftime("%Y-%m-%d")

    new_account_info = {
        "username": username,
        "password": new_password if success else password,
        "update_date": update_date,
        "success": success
    }
    new_account_info.update(v)
    result_db_list[k] = new_account_info

md_db_list.update(result_db_list)
sorted_keys = sorted(md_db_list.keys())

# 테이블 헤더 생성
table_md = "| Name                      | Host                                                                                               | Port | Username | Password   | Last Update Date |\n"
table_md += "|---------------------------|----------------------------------------------------------------------------------------------------|------|----------|------------|------------------|\n"

# 각 딕셔너리의 값을 테이블 형식으로 변환
for k in sorted_keys:
    v = md_db_list[k]
    host = v["host"]
    port = v["port"]
    username = v["username"]
    password = v["password"]
    update_date = v["update_date"]
    table_md += f"| {k} | {host} | {port} | {username} | {password} | {update_date} |\n"

# 마크다운 파일로 저장
with open(md_file, "w+") as f:
    f.write(table_md)
    print("✅파일 저장 성공", md_file)


with open(md_file, "r") as f:
    md_content = f.read()
    display(Markdown(md_content))

#endregion